In [2]:
# Install necessary libraries
%pip install kaggle pandas numpy tensorflow scikit-learn



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Make a directory for Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/  # Place your Kaggle API JSON file in the current working directory

# Set permissions for the API token
!chmod 600 ~/.kaggle/kaggle.json

# Download Jigsaw dataset
!kaggle competitions download -c jigsaw-multilingual-toxic-comment-classification

# Unzip the dataset files
!unzip jigsaw-multilingual-toxic-comment-classification.zip


The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\54321\AppData\Local\Programs\Python\Python312\Scripts\kaggle.exe\__main__.py", line 4, in <module>
  File "C:\Users\54321\AppData\Local\Programs\Python\Python312\Lib\site-packages\kaggle\__init__.py", line 7, in <module>
    api.authenticate()
  File "C:\Users\54321\AppData\Local\Programs\Python\Python312\Lib\site-packages\kaggle\api\kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in C:\Users\54321\.kaggle. Or use the environment method. See setup instructions at https://github.com/Ka

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Bidirectional,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import joblib


In [5]:
# Load the Tanglish dataset
data = pd.read_csv('D:\clg work\Project Phase 2\project\merged.csv',encoding='latin1')

# Display the first few rows of the dataset
data.head()


<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\54321\AppData\Local\Temp\ipykernel_9632\3620825966.py:2: SyntaxWarning: invalid escape sequence '\c'
  data = pd.read_csv('D:\clg work\Project Phase 2\project\merged.csv',encoding='latin1')


,tweet,sentiment,emotion,class
0,arumai,0,0,0
1,pani poori ku pirandha unaku thaai mozhiyoda a...,1,1,1
2,arumai,0,0,0
3,dmk ku vote potutomnu yarelam ipa feel panitu ...,1,1,1
4,appa irukura varaikum avarum enkita pesa matar...,0,1,0


In [7]:
# We are focusing on 'tweet' and 'class' columns
data = data[['tweet', 'class']]

# Check for missing values in 'tweet' and 'class' columns
print("Missing values:\n", data.isnull().sum())

# Drop rows with missing data (if any)
data.dropna(subset=['tweet', 'class'], inplace=True)

# Ensure 'class' column is of integer type (0 for non-toxic, 1 for toxic)
data['class'] = data['class'].apply(lambda x: 1 if int(x) > 0 else 0)

# Verify the conversion
print("Unique class values: ", data['class'].unique())

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['class'], test_size=0.2, random_state=42)

# Print the shapes of train and test data to verify
print(f"Training data shape: {X_train.shape}, {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, {y_test.shape}")


Missing values:
 tweet    0
class    0
dtype: int64
Unique class values:  [0 1]
Training data shape: (804,), (804,)
Testing data shape: (201,), (201,)


In [8]:
# Tokenize and pad sequences
MAX_NUM_WORDS = 20000  # Limit vocab size
MAX_SEQUENCE_LENGTH = 100  # Max length of each tweet

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_SEQUENCE_LENGTH)
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_SEQUENCE_LENGTH)

# Save the tokenizer for future use
import pickle
with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [9]:
# Define the model architecture
model = Sequential()
model.add(Embedding(MAX_NUM_WORDS, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model summary
model.summary()


c:\Users\54321\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Train the model
history = model.fit(X_train_pad, y_train, epochs=10, batch_size=128, validation_data=(X_test_pad, y_test))


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 276ms/step - accuracy: 0.5258 - loss: 0.6929 - val_accuracy: 0.6667 - val_loss: 0.6805
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 175ms/step - accuracy: 0.6579 - loss: 0.6772 - val_accuracy: 0.6716 - val_loss: 0.6422
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 242ms/step - accuracy: 0.7046 - loss: 0.6312 - val_accuracy: 0.7463 - val_loss: 0.5746
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 281ms/step - accuracy: 0.8407 - loss: 0.4417 - val_accuracy: 0.6816 - val_loss: 0.6514
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 294ms/step - accuracy: 0.8139 - loss: 0.4582 - val_accuracy: 0.8308 - val_loss: 0.4449
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 279ms/step - accuracy: 0.9276 - loss: 0.2504 - val_accuracy: 0.8109 - val_loss: 0.4724
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 277ms/step - accuracy: 0.9493 - loss: 0.2021 - val_accuracy: 0.8308 - val_loss: 0.4256
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 272ms/step - accuracy: 0.9460 - loss: 0.1893 - val_accuracy: 0.8358 - val_loss:

In [11]:
# Import the necessary function
from sklearn.metrics import accuracy_score

# Predict on the test set
y_pred = (model.predict(X_test_pad) > 0.5).astype('int32')

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Save the model
model.save('tanglish_classification_model.h5')


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step


Accuracy: 84.08%


In [12]:
import pickle  # Import pickle module
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
#from google.colab import files  # For downloading files in Colab

# Load the saved model
loaded_model = load_model('tanglish_classification_model.h5')

# Save the tokenizer
with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle)

# Download the tokenizer.pkl file to your local system
#files.download('tokenizer.pkl')

# Load the tokenizer (when needed)
with open('tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Function to predict toxicity of a new comment
def predict_toxicity(comment):
    # Tokenize and pad the input comment
    sequence = tokenizer.texts_to_sequences([comment])
    padded_sequence = pad_sequences(sequence, maxlen=MAX_SEQUENCE_LENGTH)

    # Make a prediction
    prediction = loaded_model.predict(padded_sequence)

    # Determine if the comment is toxic (1) or not (0)
    is_toxic = prediction[0][0] > 0.5  # Assuming binary classification
    return "Toxic" if is_toxic else "Non-Toxic"



In [13]:
# Main loop to get user input
while True:\
    # Get input from the user
    new_comment = input("Enter a comment (or type 'exit' to quit): ")

    # Exit the loop if the user types 'exit'
    if new_comment.lower() == 'exit':
        break

    # Predict and print the result
    result = predict_toxicity(new_comment)
    print(f"The comment is: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step
The comment is: Non-Toxic
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
The comment is: Non-Toxic
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
The comment is: Toxic
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
The comment is: Toxic
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
The comment is: Non-Toxic
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
The comment is: Non-Toxic
